# Twitter Sentiment Analysis

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [2]:
# use this to stretch the dataframe view
pd.set_option('display.max_colwidth', None)

In [8]:
df_train = pd.read_csv('/content/test.csv')
df_test = pd.read_csv('/content/train.csv')

In [9]:
df_train.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
df_train = df_train.drop(['keyword','location'], axis=1)
df_test = df_test.drop(['keyword','location'], axis=1)

In [11]:
df_train.head()

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, stay safe everyone."
2,3,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
x = df_train.sort_values(by= ['id'], ascending=True)
x_test = df_test.sort_values(by= ['id'], ascending=True)

In [13]:
#Retrieve the text from the dataframe as a numpy array
twitts = x.loc[:,x.columns=='text'].values.flatten()
twitts_test = x_test.loc[:,x_test.columns=='text'].values.flatten()

In [14]:
# see a sample of the collected twitts:
print(twitts[:10])

['Just happened a terrible car crash'
 'Heard about #earthquake is different cities, stay safe everyone.'
 'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all'
 'Apocalypse lighting. #Spokane #wildfires'
 'Typhoon Soudelor kills 28 in China and Taiwan'
 "We're shaking...It's an earthquake"
 "They'd probably still show more life than Arsenal did yesterday, eh? EH?"
 'Hey! How are you?' 'What a nice hat?' 'Fuck off!']


In [15]:
t = Tokenizer()
def preprocess(text):
    seqs = t.fit_on_texts(text)
    return seqs

In [16]:
#fit tokenizer on training set:
tokens=preprocess(twitts)

In [17]:
#text to sequences
X_train = t.texts_to_sequences(twitts)
X_test = t.texts_to_sequences(twitts_test)

In [18]:
#pad sequences
X_train = np.array(X_train, dtype=object)
X_test = np.array(X_test, dtype=object)
X_train = keras.preprocessing.sequence.pad_sequences(X_train)
X_test = keras.preprocessing.sequence.pad_sequences(X_test)

In [19]:
# this is a single encoded twitt:
print(X_train[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0  34 567   5 985 135 100]


In [20]:
#size of the vocabulary
vocab_size = len(t.word_index)
print('Size of vocabulary:', vocab_size)

Size of vocabulary: 12818


In [22]:
# target training variable:
y_train = df_train.target.values

AttributeError: ignored

In [23]:
# the y variable represents the labels (1 for true disaster, 0 otherwise):
print(y_train[0])

NameError: ignored

In [ ]:
#create the model
model = Sequential()
model.add(Embedding(input_dim = vocab_size+2, output_dim = 32, name='Embedding'))
model.add(LSTM(64)) 
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding (Embedding)        (None, None, 32)          726464    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 755,521
Trainable params: 755,521
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, shuffle=True, epochs=10, batch_size=64)

Epoch 1/10


2022-01-06 01:47:06.053583: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


119/119 [==============================] - 7s 40ms/step - loss: 0.5811 - accuracy: 0.6917
Epoch 2/10
119/119 [==============================] - 5s 41ms/step - loss: 0.3157 - accuracy: 0.8748
Epoch 3/10
119/119 [==============================] - 4s 38ms/step - loss: 0.1637 - accuracy: 0.9419
Epoch 4/10
119/119 [==============================] - 4s 38ms/step - loss: 0.0888 - accuracy: 0.9668
Epoch 5/10
119/119 [==============================] - 5s 40ms/step - loss: 0.0505 - accuracy: 0.9837
Epoch 6/10
119/119 [==============================] - 5s 38ms/step - loss: 0.0353 - accuracy: 0.9901
Epoch 7/10
119/119 [==============================] - 4s 38ms/step - loss: 0.0305 - accuracy: 0.9917
Epoch 8/10
119/119 [==============================] - 4s 37ms/step - loss: 0.0300 - accuracy: 0.9921
Epoch 9/10
119/119 [==============================] - 5s 42ms/step - loss: 0.0260 - accuracy: 0.9926
Epoch 10/10
119/119 [==============================] - 4s 38ms/step - loss: 0.0225 - accuracy: 0.9934


In [ ]:
train_score = model.evaluate(X_train, y_train)

238/238 [==============================] - 3s 9ms/step - loss: 0.0151 - accuracy: 0.9961


In [ ]:
preds = np.where(model.predict(X_test)>0.5,1.0,0.0).flatten().astype('int32')

In [ ]:
results = {'text':twitts_test, 'target':preds}
df_results = pd.DataFrame(results)
df_results.head()

,text,target
0,Just happened a terrible car crash,1
1,"Heard about #earthquake is different cities, stay safe everyone.",0
2,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",1
3,Apocalypse lighting. #Spokane #wildfires,1
4,Typhoon Soudelor kills 28 in China and Taiwan,1


In [ ]:
submission = pd.merge(df_test, df_results, on='text').drop_duplicates()
submission.drop(['text'],axis=1).to_csv('submission.csv', index=False)